#  Simple program to output rows per vertice, edge


### Setup: Open graph

In [ ]:
import os

from katana import remote

os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

# my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)

### Run Analytics:

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:
import katana_enterprise


#  Also at,
#     http://localhost:8080/v2/widgets/katana-frontend

my_client.widgets().operations()
# my_client.widgets().databases()

# my_client.widgets().dashboard()                #  No-op currently


In [ ]:
# ##################################################################

In [ ]:
#  LIST ALL VERTICES WITH NODE COUNTS
#
#  From,
#     https://stackoverflow.com/questions/23334688/how-to-get-different-labels-and-their-count-in-neo4j-by-cypher-query


result1 = my_graph.query("""

   MATCH (a) WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """)

display(print(result1))


result2 = my_graph.query("""

   MATCH (a) 
   WITH COUNT(a) AS tempCnt
   RETURN tempCnt
   
   """)

display(print(result2))


In [ ]:
#  LIST ALL EDGES WITH COUNTS

result1 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """)

display(print(result1))


result2 = my_graph.query("""

   MATCH (m)-[r]->(n) 
   WITH COUNT(r) AS tempCnt
   RETURN tempCnt

   """)

display(print(result2))


In [ ]:
# ##################################################################

In [ ]:

#  Delete all of the Edges from Claimant --> Payment


l_result = my_graph.query("""
   MATCH (p: payment) - [e: WAS_PAID] -> (c: claimant)
   RETURN count(e)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (p: payment) - [e: WAS_PAID] -> (c: claimant)
   DELETE e
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (p: payment) - [e: WAS_PAID] -> (c: claimant)
   RETURN count(e)
   """)
display(print(l_result))


In [ ]:

#  Get all Payments, requirement to operate below

l_result1 = my_graph.query("""

   MATCH (p: payment)
   RETURN p.id as pid, p.amount as amt

   """)

display(print(l_result1[0:5]))


In [ ]:

#  Singleton test only 

l_amt = 117.71
l_pid = "P_2467"
   #
l_cid = "C_" + l_pid[2:]

l_query = """

   MATCH (p: payment ) WHERE p.id = '{0}'
   RETURN p.id

   """.format(l_pid)

l_result7 = my_graph.query(l_query)
   #
display(print(l_result7[0:5]))


l_query = """

   MATCH (c: claimant ) WHERE c.id = '{0}'
   RETURN c.id

   """.format(l_cid)

l_result8 = my_graph.query(l_query)
   #
display(print(l_result8[0:5]))




In [ ]:

#  Singleton test only; works, fugly or not, Why can't I put this in a loop ?

l_amt = 117.71
l_pid = "P_2874"
   #
l_cid = "C_" + l_pid[2:]

l_query = """

   MATCH (p: payment ) WHERE p.id = '{0}'
   MATCH (c: claimant) WHERE c.id = '{1}'
   CREATE (p) -[r:WAS_PAID]-> (c)
   SET r.amount = {2}

   """.format(l_pid, l_cid, l_amt)

l_result0 = my_graph.query(l_query)

print("--")


In [ ]:
# ##################################################################

In [ ]:

#  Throws error in cell below; works with a break in the loop after first invocation

#  Create the Edge records, Payment -> Claimant
#
#  This is fugly; there is a means to do this with straight OpenCypher, couldn't get it to work ..


for l_each in l_result1.iterrows():
   l_amt = l_each[1][0]
   l_pid = l_each[1][1]
      #
   l_cid = "C_" + l_pid[2:]
    
   l_query = """

      MATCH (p: payment ) WHERE p.id = '{0}'
      MATCH (c: claimant) WHERE c.id = '{1}'
      CREATE (p) -[r:WAS_PAID]-> (c)
      SET r.amount = {2}
   
      """.format(l_pid, l_cid, l_amt)

   l_result0 = my_graph.query(l_query)
      #
   print(".", end='')
   # break
   

print("  ")
print("  ")


In [ ]:
# ##################################################################

In [ ]:

#  From a Google ..  FOREACH doesn't work currently ..

MATCH (e:Employees)
OPTIONAL MATCH (m:Manager)
WHERE e.name = m.name
WITH e, COLLECT(m) AS mList

FOREACH(x IN mList | CREATE (x)-[:REL_NAME]->(e))

WITH e
OPTIONAL MATCH (a:Address)
WHERE e.zipcode = a.zipcode
WITH e, COLLECT(a) AS aList
FOREACH(y IN aList | CREATE (e)-[:ADDR_REL]->(y))



In [ ]:

#  FOREACH doesn't work currently ..

l_resultX = my_graph.query("""

   MATCH (p: payment)
   WITH COLLECT (p) AS pa
   
   FOREACH (x IN pa | CREATE (x: payment2))

   """)

print("--")



In [ ]:
# ##################################################################

In [ ]:
#  Get all Payments, requirement to operate below

edge_fields2 = ['START_ID', 'END_ID', 'AMOUNT']


l_result1 = my_graph.query("""

   MATCH (p: payment)
   RETURN p.id as pid, p.amount as amt

   """)

display(print(l_result1[0:5]))

In [ ]:

import csv

i_outputfile      = "e_payment_claimant4.csv"
payment_claimants = []

for l_each in l_result1.iterrows():
   l_amt = l_each[1][0]
   l_pid = l_each[1][1]
      #
   l_cid = "C_" + l_pid[2:]

   payment_claimant = dict()
      #
   payment_claimant['START_ID'] = l_pid
   payment_claimant['END_ID'  ] = l_cid
   payment_claimant['AMOUNT'  ] = l_amt
      #
   payment_claimants.append(payment_claimant)


with open (i_outputfile, "w") as csvfile:
   l_writer = csv.DictWriter(csvfile, fieldnames=edge_fields2, delimiter="|")
      #
   l_writer.writeheader()
   l_writer.writerows(payment_claimants)

    
print("--")


In [ ]:
#  Actual graph load, edge only

from katana.remote import import_data


l_edge="gs://farrell-data-bucket/AA_Anand_Insurance/45_EdgeHeader4.txt"
   #
l_dir="gs://farrell-data-bucket/AA_Anand_Insurance/"

import_data.csv(
   my_graph,
      #
   input_edge_path     = l_edge,
   default_id_property = "id",
   ids_are_integers    = False,
   operation           = import_data.Operation.Insert,
      #
   input_dir           = l_dir,
   data_delimiter      = "|",
   schema_delimiter    = "|",
   files_have_headers  = True,
   # partition_policy = "random-oec"
   )

print ("--")

